In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

plt.style.use("seaborn-v0_8")


# Control cinemático del manipulador PR planar

Este cuaderno resume el flujo completo para modelar, analizar y simular el control cinemático de un robot planar con una junta prismática seguida de una rotacional.

Cada sección está pensada para reutilizar las funciones y facilitar su extensión a nuevos escenarios.

## 1. Parámetros del robot y simulación

In [ ]:
# Geometría del robot (L1 corresponde al offset fijo del tramo prismático)
params_pr = {
    'L1': 2.0,
    'L2': 2.0
}

# Ganancias y discretización temporal
sim_config = {
    'kp': np.diag([1.5, 1.5]),
    't0': 0.0,
    'tf': 10.0,
    'n_steps': 600
}

def time_vector(cfg):
    t = np.linspace(cfg['t0'], cfg['tf'], cfg['n_steps'])
    dt = t[1] - t[0]
    return t, dt

t, dt = time_vector(sim_config)


## 2. Cinemática directa

In [ ]:
def fkine_pr(q, params):
    """Cinemática directa para el robot PR.

    Parameters
    ----------
    q : array-like, shape (2,)
        q[0] desplazamiento prismático, q[1] ángulo rotacional.
    params : dict con claves 'L1' y 'L2'

    Returns
    -------
    np.ndarray, shape (2,)
        Coordenadas cartesianas del efector final (x, y).
    """
    q1, q2 = q
    L1, L2 = params['L1'], params['L2']
    x = L1 + q1 + L2 * np.cos(q2)
    y = L2 * np.sin(q2)
    return np.array([x, y])

# Ejemplo numérico
fkine_pr(np.array([0.2, np.pi/4]), params_pr)


## 3. Jacobiano analítico

In [ ]:
def jacobian_pr(q, params):
    q1, q2 = q
    L2 = params['L2']
    return np.array([
        [1.0, -L2 * np.sin(q2)],
        [0.0,  L2 * np.cos(q2)]
    ])

jacobian_pr(np.array([0.0, 0.0]), params_pr)


### Derivación simbólica

In [ ]:
q1_s, q2_s, L1_s, L2_s = sp.symbols('q1 q2 L1 L2')
px = L1_s + q1_s + L2_s * sp.cos(q2_s)
py = L2_s * sp.sin(q2_s)
f_pr = sp.Matrix([px, py])
J_pr = f_pr.jacobian([q1_s, q2_s])
sp.simplify(J_pr)


## 4. Trayectoria de referencia

In [ ]:
def circular_task_reference(radius=1.0, center=(3.5, 0.0), omega=2*np.pi, t=None):
    if t is None:
        raise ValueError('Se requiere vector de tiempo')
    cx, cy = center
    xd = cx + radius * np.cos(omega * t)
    yd = cy + radius * np.sin(omega * t)
    xdp = -radius * omega * np.sin(omega * t)
    ydp =  radius * omega * np.cos(omega * t)
    return np.vstack((xd, yd)), np.vstack((xdp, ydp))

Xd, Vd = circular_task_reference(radius=1.0, center=(3.5, 0.0), omega=2*np.pi, t=t)
Xd[:, :3]


## 5. Control cinemático

In [ ]:
def kinematic_control_step(q, params, xd, xdp, kp):
    x = fkine_pr(q, params)
    J = jacobian_pr(q, params)
    error = xd - x
    omega = np.linalg.solve(J, xdp + kp @ error)
    return omega, x, error

_demo_q = np.array([0.0, 0.0])
omega_demo, x_demo, e_demo = kinematic_control_step(
    _demo_q, params_pr, Xd[:, 0], Vd[:, 0], sim_config['kp'])
omega_demo, x_demo, e_demo


## 6. Simulación discreta

In [ ]:
def simulate_pr_control(q0, params, Xd, Vd, t, kp):
    n = t.size
    dt = t[1] - t[0]
    q = np.zeros((2, n))
    X = np.zeros((2, n))
    q[:, 0] = q0
    X[:, 0] = fkine_pr(q0, params)
    for k in range(1, n):
        omega, _, _ = kinematic_control_step(q[:, k-1], params, Xd[:, k-1], Vd[:, k-1], kp)
        q[:, k] = q[:, k-1] + dt * omega
        X[:, k] = fkine_pr(q[:, k], params)
    return q, X

q0 = np.array([0.0, 0.0])
theta_pr, X_pr = simulate_pr_control(q0, params_pr, Xd, Vd, t, sim_config['kp'])
X_pr[:, :3]


## 7. Visualización

In [ ]:
def plot_tracking(X, Xd, t):
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))

    ax[0].plot(Xd[0], Xd[1], '--', label='Deseada')
    ax[0].plot(X[0], X[1], label='Seguimiento')
    ax[0].set_aspect('equal', adjustable='box')
    ax[0].set_xlabel('x [m]')
    ax[0].set_ylabel('y [m]')
    ax[0].legend()
    ax[0].set_title('Trayectorias cartesianas')

    ax[1].plot(t, Xd[0] - X[0], label='Error en x')
    ax[1].plot(t, Xd[1] - X[1], label='Error en y')
    ax[1].set_xlabel('Tiempo [s]')
    ax[1].set_ylabel('Error [m]')
    ax[1].legend()
    ax[1].set_title('Errores de seguimiento')

    fig.tight_layout()
    return fig

plot_tracking(X_pr, Xd, t)


### Animación

In [ ]:
def animate_pr(theta, X, Xd, t, params, axis_len=0.5, trail=True, skip=2):
    L1, L2 = params['L1'], params['L2']
    fig, ax = plt.subplots()
    ax.set_aspect('equal', adjustable='box')
    margin = 0.6
    x_all = np.concatenate([X[0], Xd[0], np.array([0.0])])
    y_all = np.concatenate([X[1], Xd[1], np.array([0.0])])
    ax.set_xlim(np.min(x_all) - margin, np.max(x_all) + margin)
    ax.set_ylim(np.min(y_all) - margin, np.max(y_all) + margin)
    ax.set_xlabel('x [m]')
    ax.set_ylabel('y [m]')
    ax.set_title('Robot PR planar')

    desired_line, = ax.plot(Xd[0], Xd[1], '--', linewidth=1.2, label='Trayectoria deseada')
    trail_line,   = ax.plot([], [], ':', linewidth=1.5, label='Rastro')
    link1_line, = ax.plot([], [], 'k-', linewidth=3)
    link2_line, = ax.plot([], [], 'k-', linewidth=3)
    ee_point,   = ax.plot([], [], 'o', markersize=5)

    f0_x, = ax.plot([], [], color='r', linewidth=2)
    f0_y, = ax.plot([], [], color='g', linewidth=2)
    f1_x, = ax.plot([], [], color='r', linewidth=2)
    f1_y, = ax.plot([], [], color='g', linewidth=2)
    f2_x, = ax.plot([], [], color='r', linewidth=2)
    f2_y, = ax.plot([], [], color='g', linewidth=2)
    base_dot,  = ax.plot([], [], 'o', color='b')
    joint_dot, = ax.plot([], [], 'o', color='b')
    ee_dot,    = ax.plot([], [], 'o', color='b')
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
    ax.legend(loc='upper right')

    def frame_axes(origin, angle):
        c, s = np.cos(angle), np.sin(angle)
        p = np.array(origin)
        x_axis = p + axis_len * np.array([c, s])
        y_axis = p + axis_len * np.array([-s, c])
        return p, x_axis, y_axis

    def init():
        trail_line.set_data([], [])
        for ln in (link1_line, link2_line, ee_point,
                   f0_x, f0_y, f1_x, f1_y, f2_x, f2_y):
            ln.set_data([], [])
        for dot in (base_dot, joint_dot, ee_dot):
            dot.set_data([], [])
        time_text.set_text('')
        return (
            link1_line, link2_line, ee_point, trail_line,
            f0_x, f0_y, f1_x, f1_y, f2_x, f2_y,
            base_dot, joint_dot, ee_dot, time_text
        )

    def update(frame):
        idx = min(frame * skip, t.size - 1)
        q1, q2 = theta[:, idx]
        O0 = np.array([0.0, 0.0])
        O1 = np.array([L1 + q1, 0.0])
        O2 = np.array([L1 + q1 + L2 * np.cos(q2), L2 * np.sin(q2)])

        link1_line.set_data([O0[0], O1[0]], [O0[1], O1[1]])
        link2_line.set_data([O1[0], O2[0]], [O1[1], O2[1]])
        ee_point.set_data([O2[0]], [O2[1]])

        if trail:
            trail_line.set_data(X[0, :idx+1], X[1, :idx+1])

        p0, p0x, p0y = frame_axes(O0, 0.0)
        f0_x.set_data([p0[0], p0x[0]], [p0[1], p0x[1]])
        f0_y.set_data([p0[0], p0y[0]], [p0[1], p0y[1]])
        base_dot.set_data([O0[0]], [O0[1]])

        p1, p1x, p1y = frame_axes(O1, 0.0)
        f1_x.set_data([p1[0], p1x[0]], [p1[1], p1x[1]])
        f1_y.set_data([p1[0], p1y[0]], [p1[1], p1y[1]])
        joint_dot.set_data([O1[0]], [O1[1]])

        p2, p2x, p2y = frame_axes(O2, q2)
        f2_x.set_data([p2[0], p2x[0]], [p2[1], p2x[1]])
        f2_y.set_data([p2[0], p2y[0]], [p2[1], p2y[1]])
        ee_dot.set_data([O2[0]], [O2[1]])

        time_text.set_text(f't = {t[idx]:.2f} s')
        return (
            link1_line, link2_line, ee_point, trail_line,
            f0_x, f0_y, f1_x, f1_y, f2_x, f2_y,
            base_dot, joint_dot, ee_dot, time_text
        )

    interval_ms = max(10, int(dt * 1000))
    frames = int(np.ceil(t.size / skip))
    anim = FuncAnimation(fig, update, frames=frames, init_func=init,
                         blit=True, interval=interval_ms)
    plt.close(fig)
    return anim

anim_pr = animate_pr(theta_pr, X_pr, Xd, t, params_pr, axis_len=0.5, trail=True, skip=2)
HTML(anim_pr.to_jshtml())
